## Preparation
- Install packages
- Mount notebook to google drive
- Import package and functions

In [1]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
# Mount into drive

from google.colab import drive

drive.mount("/content/drive")

%cd '/content/drive/MyDrive/Adapter/'

Mounted at /content/drive
/content/drive/MyDrive/Adapter


In [4]:
import dataload
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
from trainer import BertTrainer
from lora import (
    LinearLoRA,
    create_lora,
    add_lora_layers,
    freeze_model,
    unfreeze_model,
    create_linear,
    merge_lora_layers,
)
import torch
from pathlib import Path

## Load Data, Split to train-validation-test

In [6]:
train_loader, val_loader, test_loader = dataload.load_data()

# load tokenizer and pretrained model
tokenizer_base = BertTokenizer.from_pretrained('bert-base-uncased')

config = BertConfig.from_pretrained('bert-base-uncased', num_labels=3)

# Load the BERT model for sequence classification with the specified configuration
bert_base = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    config=config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Bert Tuning

In [ ]:
#bert base
trainer_bert_base = BertTrainer(
    bert_base,
    tokenizer_base,
    lr=1e-04,
    epochs=10,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    output_dir='./results/bert_base_fine_tuned',
    output_filename='bert_base_lr1e-4',
    save=True,
)

trainer_bert_base.train(evaluate=True)

EP (train) 0: 100%|| 53/53 [00:32<00:00,  1.63it/s]

samples=3392,             correct=2822,             acc=0.832,             recall=0.802,             prec=0.8175,             f1=0.8083,             loss=0.4087



EP (eval) 0: 100%|| 12/12 [00:03<00:00,  3.22it/s]


samples=727,             correct=609,             acc=0.8377,             recall=0.8029,             prec=0.836,             f1=0.8122,             loss=0.3779


EP (train) 1: 100%|| 53/53 [00:28<00:00,  1.89it/s]

...

samples=727,             correct=621,             acc=0.8542,             recall=0.8435,             prec=0.8412,             f1=0.8398,             loss=0.3502



EP (train) 9: 100%|| 53/53 [00:28<00:00,  1.85it/s]

samples=3392,             correct=3133,             acc=0.9236,             recall=0.9215,             prec=0.9232,             f1=0.9223,             loss=0.1943



EP (eval) 9: 100%|| 12/12 [00:03<00:00,  3.68it/s]

samples=727,             correct=625,             acc=0.8597,             recall=0.8523,             prec=0.842,             f1=0.8462,             loss=0.3625


## Lora Tuning

In [7]:
add_lora_layers(bert_base, r=10, lora_alpha=16)  # inject the LoRA layers into the model
freeze_model(bert_base)  # freeze the non-LoRA parameters

n_params = 0
n_trainable_params = 0

# count the number of trainable parameters
for n, p in bert_base.named_parameters():
    n_params += p.numel()
    if p.requires_grad:
        n_trainable_params += p.numel()

print(f"Total parameters: {n_params}")
print(f"Trainable parameters: {n_trainable_params}")
print(f"Percentage trainable: {round(n_trainable_params / n_params * 100, 2)}%")

#bert base lora all r = 10
trainer_bert_base_lora = BertTrainer(
    bert_base,
    tokenizer_base,
    lr=1e-03,
    epochs=5,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    output_dir='./results/bert_base_fine_tuned_lora_r10',
    output_filename='bert_base_lora_r10_lr1e-3',
    save=True,
)

trainer_bert_base_lora.train(evaluate=True)

Total parameters: 109853187
Trainable parameters: 370947
Percentage trainable: 0.34%


EP (train) 0: 100%|| 53/53 [00:29<00:00,  1.79it/s]

samples=3392,             correct=2278,             acc=0.6716,             recall=0.4921,             prec=0.6135,             f1=0.5106,             loss=0.75



EP (eval) 0: 100%|| 12/12 [00:02<00:00,  4.04it/s]


samples=727,             correct=597,             acc=0.8212,             recall=0.7922,             prec=0.7858,             f1=0.7889,             loss=0.447


...

samples=727,             correct=618,             acc=0.8501,             recall=0.8407,             prec=0.835,             f1=0.8291,             loss=0.4154



EP (train) 4: 100%|| 53/53 [00:27<00:00,  1.94it/s]

samples=3392,             correct=3212,             acc=0.9469,             recall=0.9416,             prec=0.9446,             f1=0.9431,             loss=0.1379



EP (eval) 4: 100%|| 12/12 [00:03<00:00,  3.48it/s]

samples=727,             correct=622,             acc=0.8556,             recall=0.8641,             prec=0.8461,             f1=0.8535,             loss=0.4239


In [10]:
state_dict = torch.load("results/bert_base_fine_tuned_lora_r10/bert_base_lora_r10_lr1e-3_epoch_2.pt")
bert_base.load_state_dict(state_dict["model_state_dict"])

# merge weights
merge_lora_layers(bert_base)
unfreeze_model(bert_base)

# create directory and filepaths
dir_path = Path("results/bert_base_fine_tuned_lora_r10/merged")
dir_path.mkdir(parents=True, exist_ok=True)
file_path = dir_path / "bert_base_lora_r10_epoch_1_merged.pt"

# save model
torch.save({
    "model_state_dict": bert_base.state_dict(),
}, file_path)

trainer_bert_base_lora_r8 = BertTrainer(
    bert_base,
    tokenizer_base,
    lr=5e-06,
    epochs=5,
    train_dataloader=train_loader,
    eval_dataloader=test_loader,
    output_dir='./results/bert_base_fine_tuned_lora_r10',
    output_filename='bert_base_lora_r10',
    save=False,
)

trainer_bert_base_lora_r8.evaluate()

EP (eval) 0: 100%|| 12/12 [00:02<00:00,  4.02it/s]

samples=727,             correct=609,             acc=0.8377,             recall=0.7936,             prec=0.8419,             f1=0.8124,             loss=0.3656


In [14]:
import copy
test = copy.deepcopy(trainer_bert_base_lora)
test.eval_dataloader = test_loader
test.evaluate()

EP (eval) 0: 100%|| 12/12 [00:03<00:00,  3.54it/s]

samples=727,             correct=609,             acc=0.8377,             recall=0.7936,             prec=0.8419,             f1=0.8124,             loss=0.3656
